# 2. Web Scraping - BeautifulSoup

<img src='https://xn--kvin-duranty-beb.fr/wp-content/uploads/2022/10/Web-Scraping-_-IPSSI-PRS-1.png'>

Dans cet exercice, nous utiliserons la bibliothèque BeautifulSoup afin de collecter les données des sites internet suivants :

- Partie 1 : [TheProject.html](http://info.cern.ch/hypertext/WWW/TheProject.html)
Nous reprenons le site scrapé  précédement via la bibliothèque `requests`, l'objectif est de comprendre le fonctionnement de la bibliothèque `BeautifulSoup`.


- Partie 2 : [Blog Du Modérateur](https://www.blogdumoderateur.com/)
L'objectif sera de collecter la liste des articles présents sur la page d'accueil du site.

- Partie 3 : [Doctolib](https://www.doctolib.fr/dentiste/paris)
Nous verrons que certains sites ont une structure html/css qui entrave la collecte de données, heureusement d'autres bibliothèques existent et permettent de contourner ces restrictions (Selenium).



# Partie 1 - [TheProject.html](http://info.cern.ch/hypertext/WWW/TheProject.html)

## 2.1 Installez la bibliothèque BeautifulSoup.
Utilitisez la commande suivante dans votre terminal :  `pip install beautifulsoup4`

## 2.2 Importez le package `BeautifulSoup` à partir de la bibliothèque `bs4`.
Importez également la bibliothèque `request`

In [2]:
#pip install beautifulsoup4
import requests
from bs4 import BeautifulSoup

## 2.3 Faites une requête à l'adresse [TheProject.html](http://info.cern.ch/hypertext/WWW/TheProject.html) via la bibliothèque `requests`.
Stockez la réponse dans une variable `response` et affichez le type de cette variable.

In [5]:
response = requests.get("http://info.cern.ch/hypertext/WWW/TheProject.html")
print(type(response))

<class 'requests.models.Response'>


## 2.4 Instanciez une variable soup à partir du module `BeautifulSoup` et de la variable `response.text`.

Affichez le type de cette variable `soup` ainsi que son instance `text`

In [6]:
soup = BeautifulSoup(response.text, 'html.parser')
print(type(soup), soup.text[:100])


<class 'bs4.BeautifulSoup'> 
The World Wide Web project



World Wide WebThe WorldWideWeb (W3) is a wide-area
hypermedia informa


## 2.5 Utilisez la méthode `find_all` afin de trouver les balises `header`,`body`, `title`, `h1`, `p` et `a`.

In [19]:
# header
header_tags = soup.find_all('header')
print(f"header: {len(header_tags)} instances")

header: 1 instances


In [20]:
# body
body_tags = soup.find_all('body')
print(f"body: {len(body_tags)} instances")

body: 1 instances


In [21]:
# title
title_tags = soup.find_all('title')
print(f"title: {len(title_tags)} instances")

title: 1 instances


In [22]:
# h1
h1_tags = soup.find_all('h1')
print(f"h1: {len(h1_tags)} instances")

h1: 1 instances


In [23]:
#p
p_tags = soup.find_all('p')
print(f"p: {len(p_tags)} instances")

p: 1 instances


In [24]:
# a
a_tags = soup.find_all('a')
print(f"a: {len(a_tags)} instances")

a: 25 instances


## 2.6 Affichez les attributs du premier lien.
Récupérer dans une liste l'ensemble des attributs `href`.

In [25]:
first_link = soup.find('a')
if first_link:
    print("Attributs du premier lien :", first_link.attrs)

# Récupérer tous les attributs href dans une liste
hrefs = [link.get('href') for link in soup.find_all('a') if link.get('href') is not None]
print("Liste des attributs href :", hrefs)

Attributs du premier lien : {'name': '0', 'href': 'WhatIs.html'}
Liste des attributs href : ['WhatIs.html', 'Summary.html', 'Administration/Mailing/Overview.html', 'Policy.html', 'News/9211.html', 'FAQ/List.html', '../DataSources/Top.html', '../DataSources/bySubject/Overview.html', '../DataSources/WWW/Servers.html', 'Help.html', 'Status.html', 'LineMode/Browser.html', 'Status.html#35', 'NeXT/WorldWideWeb.html', 'Daemon/Overview.html', 'Tools/Overview.html', 'MailRobot/Overview.html', 'Status.html#57', 'Technical.html', 'Bibliography.html', 'People.html', 'History.html', 'Helping.html', '../README.html', 'LineMode/Defaults/Distribution.html']


# Partie 2 - Collecte de données du [Blog du Modérateur](https://www.blogdumoderateur.com/)

<img src='https://f.hellowork.com/blogdumoderateur/2017/05/blogdumoderateur-og.png'>

Ce deuxième exercice a pour but de collecter les articles du site le Blog Du Modérateur. Pour cela nous utiliserons la bibliothèque BeautifulSoup afin de scraper les éléments suivants présents sur la page d'accueil :

- le nom de l’article,
- le lien de l’image,
- le lien de l’article
- la catégorie de l’article
- la date de publication de l’article

Nous créerons un script qui automatisera la collecte des données en prenant en compte les exceptions et qui retournera un fichier json contenant l'ensemble de nos données.

## 2.7 Faites une requête au site du Blog du Modérateur puis instanciez une variable `soup`.

Utilisez l'adresse url suivante : `https://www.blogdumoderateur.com/`

In [9]:
response_bdm = requests.get("https://www.blogdumoderateur.com/")
soup_bdm = BeautifulSoup(response_bdm.text, 'html.parser')
soup_bdm

<!DOCTYPE html>

<html lang="fr-FR">
<head><meta charset="utf-8"/><script>if(navigator.userAgent.match(/MSIE|Internet Explorer/i)||navigator.userAgent.match(/Trident\/7\..*?rv:11/i)){var href=document.location.href;if(!href.match(/[?&]nowprocket/)){if(href.indexOf("?")==-1){if(href.indexOf("#")==-1){document.location.href=href+"?nowprocket=1"}else{document.location.href=href.replace("#","?nowprocket=1#")}}else{if(href.indexOf("#")==-1){document.location.href=href+"&nowprocket=1"}else{document.location.href=href.replace("#","&nowprocket=1#")}}}}</script><script>class RocketLazyLoadScripts{constructor(){this.v="1.2.4",this.triggerEvents=["keydown","mousedown","mousemove","touchmove","touchstart","touchend","wheel"],this.userEventHandler=this._triggerListener.bind(this),this.touchStartHandler=this._onTouchStart.bind(this),this.touchMoveHandler=this._onTouchMove.bind(this),this.touchEndHandler=this._onTouchEnd.bind(this),this.clickHandler=this._onClick.bind(this),this.interceptedClicks=[],

## 2.8 Collectez dans une variable `article` les articles présents sur la page d'acceuil.

In [10]:
articles = soup_bdm.find_all('article')
print(f"Nombre d'articles trouvés : {len(articles)}")


Nombre d'articles trouvés : 54


## 2.9 Récupérez le `nom` du premier article.

In [33]:
if articles and articles[0].find('h3'):
    first_article_name = articles[0].find('h3').text.strip()
    print(first_article_name)
else:
    print("Le premier article n'a pas de balise <h2> ou aucun article trouvé.")


Microsoft Copilot est disponible sur iPhone : comment y accéder


##  2.10 Récupérez le `lien de l'image` du premier article.

In [31]:
first_article_img_link = articles[0].find('img')['data-lazy-src']
print(first_article_img_link)


https://f.hellowork.com/blogdumoderateur/2024/01/microsoft-copilot-app-ios-276x144.jpg


## 2.11 Récupérez le `lien` du premier article.

In [13]:
first_article_link = articles[0].find('a', href=True)['href']
print(first_article_link)


https://www.blogdumoderateur.com/microsoft-copilot-disponible-iphone-comment-acceder/


## 2.12 Récupérez la `catégorie` du premier article.

In [28]:

if articles:
    first_article = articles[0]
    category_element = first_article.find('a', rel='category tag')
    if category_element:
        first_article_category = category_element.text.strip()
        print(first_article_category)
    else:
        print("Aucune catégorie trouvée pour le premier article.")
else:
    print("Aucun article trouvé.")

Aucune catégorie trouvée pour le premier article.


## 2.13 Récupérez la `date de publication` du premier article.

In [27]:

first_article_date = articles[0].find('time')['datetime']
print(first_article_date)


Aucune catégorie trouvée pour le premier article.


# Créez une fonction `scraping_bdm` qui collecte l'ensemble des données des articles présent sur la page d'acceuil et qui renvoi un fichier au format `json`.

In [35]:
import requests
from bs4 import BeautifulSoup
import json

def scraping_bdm(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    articles = soup.find_all('article')
    
    articles_data = []
    
    for article in articles:
        title = article.h3.text
        link = article.find('a', href=True)['href'] if article.find('a', href=True) else "No Link"
        try :
            image = articles[-1].find('img')['data-lazy-src']
        except :
            image = None
        articles_data.append({
            'title': title,
            'link': link,
            'image': image
        })
    
    with open('articles_data.json', 'w', encoding='utf-8') as f:
        json.dump(articles_data, f, ensure_ascii=False, indent=4)

    return articles_data  

scraping_bdm("https://www.blogdumoderateur.com/")


[{'title': 'Microsoft Copilot est disponible sur iPhone\xa0: comment y accéder',
  'link': 'https://www.blogdumoderateur.com/microsoft-copilot-disponible-iphone-comment-acceder/',
  'image': None},
 {'title': 'GitHub Copilot Chat\xa0: l’assistant IA basé sur GPT-4 est disponible pour tous',
  'link': 'https://www.blogdumoderateur.com/github-copilot-chat-disponible-pour-tous/',
  'image': None},
 {'title': 'iOS 17.3\xa0: date de sortie et nouveautés à venir sur votre iPhone',
  'link': 'https://www.blogdumoderateur.com/ios-17-3-date-sortie-nouveautes-iphone/',
  'image': None},
 {'title': 'Instagram\xa0: comment changer votre nom d’utilisateur',
  'link': 'https://www.blogdumoderateur.com/instagram-comment-changer-nom-utilisateur/',
  'image': None},
 {'title': 'Les 10 actualités du digital qu’il ne fallait pas manquer en décembre 2023',
  'link': 'https://www.blogdumoderateur.com/10-actualites-digital-decembre-2023/',
  'image': None},
 {'title': 'Le chargeur universel et le port USB-C

In [38]:
import requests
from bs4 import BeautifulSoup

def scraping_bdm(url:str='https://www.blogdumoderateur.com/') -> dict:
    response_bdm = requests.get(url)
    soup_bdm = BeautifulSoup(response_bdm.text)
    artilces = soup_bdm.find_all('article')

    data = {}

    for artilce in artilces:
        
        try:image_link = artilce.find('img')['data-lazy-src'] # Image
        except:image_link = None
        
        title = artilce.h3.text                                 # Title

        try:link = artilce.find('a')['href']                    # Link
        except:link = artilce.parent['href']

        time = artilce.time['datetime'].split('T')[0]           # Time
       
        try:label = artilce.find('span', 'favtag color-b').text  # label
        except:
            try:label = artilce.parent.parent.parent.parent.h2.text
            except:label = None

        data[artilce['id']] = {
            'title' : title,
            'image' : image_link,
            'link'  : link,
            'label':  label,
            'time'  : time
        }
    return data

scraping_bdm()

{'post-150576': {'title': 'Microsoft Copilot est disponible sur iPhone\xa0: comment y accéder',
  'image': 'https://f.hellowork.com/blogdumoderateur/2024/01/microsoft-copilot-app-ios-276x144.jpg',
  'link': 'https://www.blogdumoderateur.com/microsoft-copilot-disponible-iphone-comment-acceder/',
  'label': 'Microsoft',
  'time': '2024-01-02'},
 'post-150573': {'title': 'GitHub Copilot Chat\xa0: l’assistant IA basé sur GPT-4 est disponible pour tous',
  'image': 'https://f.hellowork.com/blogdumoderateur/2024/01/Copilot-Chat-Disponible-276x144.jpg',
  'link': 'https://www.blogdumoderateur.com/github-copilot-chat-disponible-pour-tous/',
  'label': 'Développement',
  'time': '2024-01-02'},
 'post-150500': {'title': 'iOS 17.3\xa0: date de sortie et nouveautés à venir sur votre iPhone',
  'image': 'https://f.hellowork.com/blogdumoderateur/2023/12/ios-17-3-nouveautes-276x144.jpg',
  'link': 'https://www.blogdumoderateur.com/ios-17-3-date-sortie-nouveautes-iphone/',
  'label': 'Apple',
  'time'

# Partie 3 - Collecte de données de Doctolib - Bonus


<img src='https://upload.wikimedia.org/wikipedia/fr/thumb/7/7f/Logo-doctolib.svg/640px-Logo-doctolib.svg.png'>


L'objectif est de collecter les données des dentistes référencés sur le sites de Doctolib exerçant à Paris à partir de l'adresse suivante : `https://www.doctolib.fr/dentiste/paris`.

Les informations que nous souhaitons collecter sont les suivantes :
- le nom du praticien
- la profession du praticien
- l'adresse du praticien
- la ville du praticien
- l'image de la fiche Doctolib du praticien

In [ ]:
url_doctolib = "https://www.doctolib.fr/dentiste/paris"

response_doctolib = requests.get(url_doctolib)
soup_doctolib = BeautifulSoup(response_doctolib.text, 'html.parser')

for practician in soup_doctolib.find_all(class_="specific-class-for-practicians"):  
    name = practician.text.strip()  
    print(name)
